In [5]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("martin_luther_king")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [55]:
def split_line(line):
    return line.split('\n')


# A.1.1
linesEN = spark_context.textFile("hdfs://192.168.1.153:9000/europarl/europarl-v7.sv-en.en")

# A.1.2
linesSWE = spark_context.textFile("hdfs://192.168.1.153:9000/europarl/europarl-v7.sv-en.sv")

# A.1.3
if(linesSWE.count() == linesEN.count()):
    print("Yes they match :)")

# A.1.4
# English partitions
print(linesEN.getNumPartitions())

# Swedish partitions
print(linesSWE.getNumPartitions())

Yes they match :)
2
3


In [56]:
# A.2.1
# lower case function 
def lower_case(line):
    lower = line.map(lambda x: x.lower())
    return lower

# lower case english
lower_english = lower_case(linesEN)

# lower case swedish
lower_swedish = lower_case(linesSWE)

# split space function 
def split_space(line):
    splitz = line.split(' ')
    return splitz

# split words english
split_english = lower_english.map(split_space)

# split words swedish
split_swedish = lower_swedish.map(split_space)

# A.2.2
print(split_english.take(10))
print(split_swedish.take(10))

# A.2.3
if(split_english.count() == split_swedish.count()):
    print("Yes they match :)")


[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

In [ ]:
# flat map [[1,2],[1,2,3]] = [1,2,1,2,3]

In [ ]:
# release the cores for another application!
# spark_context.stop()